
#  TP Optimisation II - Fonction de Rosenbrock
## **INF4127 : Optimisation numérique sans contraintes** ##

## **Université de Yaoundé 1 - Département d'Informatique** ##
 
### **Année académique 2025-2026** ###


# ---
#

#### **Objectif** 
#### Reproduire les algorithmes de descente de gradient sur la fonction de Rosenbrock : 
 
### $$f(x, y) = (1 - x)^2 + 100(y - x^2)^2$$ ###

 #### Nous comparerons deux méthodes de descente de gradient : #####
 #### 1. **Gradient à pas optimal** (méthode de plus profonde descente) #####
#### 2. **Gradient à pas fixe** (avec différentes valeurs du pas) #####


#### **1. Imports et Configuration** 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Configuration pour de meilleurs graphiques
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True

print("Bibliothèques chargées avec succès !")

#### **2. Définition de la fonction de Rosenbrock** ####

#### La fonction de Rosenbrock, aussi appelée fonction "banane" ou "vallée", est un cas test classique en optimisation.

#### **Propriétés :**
#### - Minimum global unique en $(x^*, y^*) = (1, 1)$ avec $f(1, 1) = 0$
#### - Fonction non-convexe
#### - Vallée étroite et incurvée menant au minimum
#### - Difficile à optimiser avec des méthodes simples

In [ ]:
def rosenbrock(x, y):
    """
    Calcule la valeur de la fonction de Rosenbrock.
    f(x, y) = (1 - x)² + 100(y - x²)²
    
    Le minimum global est en (1, 1) avec f(1, 1) = 0
    """
    return (1 - x)**2 + 100 * (y - x**2)**2


def gradient_rosenbrock(x, y):
    """
    Calcule le gradient de la fonction de Rosenbrock.
    
    ∇f = (∂f/∂x, ∂f/∂y)
    ∂f/∂x = -2(1 - x) - 400x(y - x²)
    ∂f/∂y = 200(y - x²)
    
    Retourne un tuple (df_dx, df_dy)
    """
    df_dx = -2 * (1 - x) - 400 * x * (y - x**2)
    df_dy = 200 * (y - x**2)
    return df_dx, df_dy


# Test de la fonction
x_test, y_test = 0, 0
print(f"f({x_test}, {y_test}) = {rosenbrock(x_test, y_test)}")
print(f"∇f({x_test}, {y_test}) = {gradient_rosenbrock(x_test, y_test)}")
print(f"f(1, 1) = {rosenbrock(1, 1)} (minimum global)")

### **3. Algorithme de Gradient à Pas Optimal**

#### Cette méthode, appelée **méthode de plus profonde descente**, calcule à chaque itération le pas optimal $s_k$ qui minimise la fonction dans la direction du gradient :

#### $$s_k = \arg\min_{s>0} f(x_k - s\nabla f(x_k))$$

#### **Propriété importante** (vue en cours) : Deux directions de descente successives sont orthogonales, ce qui cause un comportement en zigzag.


In [ ]:
def recherche_lineaire_rosenbrock(x, y, dx, dy):
    """
    Recherche linéaire approximative pour trouver le pas optimal.
    
    On cherche s qui minimise φ(s) = f(x - s*dx, y - s*dy)
    où (dx, dy) est la direction de descente (le gradient).
    
    Méthode : échantillonnage sur un intervalle puis raffinement.
    """
    # Recherche grossière sur un intervalle large
    s_values = np.linspace(0.0001, 1, 100)
    phi_values = [rosenbrock(x - s*dx, y - s*dy) for s in s_values]
    
    # Trouver le meilleur s dans l'échantillon grossier
    idx_min = np.argmin(phi_values)
    s_best = s_values[idx_min]
    
    # Raffinement autour du meilleur s trouvé
    s_min = max(0.0001, s_best - 0.01)
    s_max = s_best + 0.01
    s_values_fine = np.linspace(s_min, s_max, 50)
    phi_values_fine = [rosenbrock(x - s*dx, y - s*dy) for s in s_values_fine]
    
    idx_min_fine = np.argmin(phi_values_fine)
    s_optimal = s_values_fine[idx_min_fine]
    
    return s_optimal


def gradient_pas_optimal(x0, y0, epsilon=1e-5, max_iter=10000):
    """
    Méthode de plus profonde descente (steepest descent).
    
    À chaque itération, on cherche le pas optimal qui minimise
    f(x_k - s*∇f(x_k)) par rapport à s.
    
    Paramètres:
    -----------
    x0, y0 : float
        Point de départ
    epsilon : float
        Critère d'arrêt sur la norme du gradient
    max_iter : int
        Nombre maximum d'itérations
    
    Retourne:
    ---------
    trajectory : list of dict
        Liste contenant l'historique des itérations
    """
    # Initialisation
    x, y = x0, y0
    trajectory = []
    
    # Enregistrer le point initial
    trajectory.append({
        'iter': 0,
        'x': x,
        'y': y,
        'f': rosenbrock(x, y),
        'grad_norm': np.linalg.norm(gradient_rosenbrock(x, y)),
        's_k': None
    })
    
    for k in range(max_iter):
        # Calculer le gradient au point courant
        df_dx, df_dy = gradient_rosenbrock(x, y)
        grad_norm = np.sqrt(df_dx**2 + df_dy**2)
        
        # Test de convergence : si le gradient est suffisamment petit, on s'arrête
        if grad_norm < epsilon:
            print(f"Convergence atteinte à l'itération {k}")
            break
        
        # Recherche du pas optimal par recherche linéaire
        s_optimal = recherche_lineaire_rosenbrock(x, y, df_dx, df_dy)
        
        # Mise à jour du point
        x = x - s_optimal * df_dx
        y = y - s_optimal * df_dy
        
        # Enregistrer les informations de cette itération
        trajectory.append({
            'iter': k + 1,
            'x': x,
            'y': y,
            'f': rosenbrock(x, y),
            'grad_norm': grad_norm,
            's_k': s_optimal
        })
    
    return trajectory

print("Algorithme de gradient à pas optimal défini !")

#### **4. Algorithme de Gradient à Pas Fixe**

#### Cette méthode utilise un pas constant $s$ à chaque itération :

#### $$x_{k+1} = x_k - s\nabla f(x_k)$$

#### **Avantage** : Simple et peu coûteuse par itération

#### **Inconvénient** : Le choix du pas est critique. Un pas trop grand fait diverger, un pas trop petit ralentit la convergence.


In [ ]:
def gradient_pas_fixe(x0, y0, s_fixe, epsilon=1e-5, max_iter=100000):
    """
    Méthode de gradient à pas fixe.
    
    À chaque itération, on fait un pas de taille constante dans la direction
    de la plus forte descente : x_{k+1} = x_k - s*∇f(x_k)
    
    Paramètres:
    -----------
    x0, y0 : float
        Point de départ
    s_fixe : float
        Taille du pas (constante)
    epsilon : float
        Critère d'arrêt sur la norme du gradient
    max_iter : int
        Nombre maximum d'itérations
    
    Retourne:
    ---------
    trajectory : list of dict
        Liste contenant l'historique des itérations
    converged : bool
        True si l'algorithme a convergé, False s'il a divergé
    """
    # Initialisation
    x, y = x0, y0
    trajectory = []
    converged = True
    
    # Enregistrer le point initial
    trajectory.append({
        'iter': 0,
        'x': x,
        'y': y,
        'f': rosenbrock(x, y),
        'grad_norm': np.linalg.norm(gradient_rosenbrock(x, y))
    })
    
    for k in range(max_iter):
        # Calculer le gradient au point courant
        df_dx, df_dy = gradient_rosenbrock(x, y)
        grad_norm = np.sqrt(df_dx**2 + df_dy**2)
        
        # Test de convergence
        if grad_norm < epsilon:
            print(f"Convergence atteinte à l'itération {k}")
            break
        
        # Mise à jour avec pas fixe
        x = x - s_fixe * df_dx
        y = y - s_fixe * df_dy
        
        # Vérifier la divergence
        if not np.isfinite(x) or not np.isfinite(y) or abs(x) > 1e10 or abs(y) > 1e10:
            print(f"Divergence détectée à l'itération {k}")
            converged = False
            break
        
        # Enregistrer les informations de cette itération
        trajectory.append({
            'iter': k + 1,
            'x': x,
            'y': y,
            'f': rosenbrock(x, y),
            'grad_norm': grad_norm
        })
    
    return trajectory, converged

print("Algorithme de gradient à pas fixe défini !")


 #### **5. Fonctions de Visualisation**

In [ ]:
def plot_contours_with_trajectory(trajectory, title="Trajectoire d'optimisation"):
    """
    Visualise les courbes de niveau de la fonction de Rosenbrock
    avec la trajectoire suivie par l'algorithme d'optimisation.
    """
    # Créer une grille de points pour tracer les courbes de niveau
    x = np.linspace(-2, 2, 400)
    y = np.linspace(-1, 3, 400)
    X, Y = np.meshgrid(x, y)
    Z = rosenbrock(X, Y)
    
    # Créer la figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Tracer les courbes de niveau avec une échelle logarithmique
    levels = np.logspace(-1, 3.5, 35)
    contour = ax.contour(X, Y, Z, levels=levels, cmap='viridis', alpha=0.6)
    ax.clabel(contour, inline=True, fontsize=8, fmt='%.1f')
    
    # Extraire les coordonnées de la trajectoire
    traj_x = [point['x'] for point in trajectory]
    traj_y = [point['y'] for point in trajectory]
    
    # Tracer la trajectoire avec des points et des lignes
    ax.plot(traj_x, traj_y, 'r.-', linewidth=2, markersize=8, 
            label='Trajectoire', alpha=0.7)
    
    # Marquer le point de départ en vert
    ax.plot(traj_x[0], traj_y[0], 'go', markersize=15, 
            label=f'Départ ({traj_x[0]:.2f}, {traj_y[0]:.2f})', zorder=5)
    
    # Marquer le point d'arrivée en bleu
    ax.plot(traj_x[-1], traj_y[-1], 'bo', markersize=15,
            label=f'Arrivée ({traj_x[-1]:.4f}, {traj_y[-1]:.4f})', zorder=5)
    
    # Marquer le minimum global en rouge (étoile)
    ax.plot(1, 1, 'r*', markersize=20, 
            label='Minimum global (1, 1)', zorder=5)
    
    # Ajouter quelques numéros d'itération le long de la trajectoire
    step = max(1, len(trajectory) // 10)
    for i in range(0, len(trajectory), step):
        ax.annotate(f"{trajectory[i]['iter']}", 
                   (traj_x[i], traj_y[i]),
                   textcoords="offset points", xytext=(5,5), 
                   fontsize=8, color='darkred')
    
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_convergence(trajectory, title="Convergence de l'algorithme"):
    """
    Trace l'évolution de la valeur de la fonction objectif
    et de la norme du gradient au cours des itérations.
    """
    iterations = [point['iter'] for point in trajectory]
    f_values = [point['f'] for point in trajectory]
    grad_norms = [point['grad_norm'] for point in trajectory]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Graphique 1 : Évolution de f(x, y)
    ax1.semilogy(iterations, f_values, 'b-', linewidth=2)
    ax1.set_xlabel('Itération k', fontsize=12)
    ax1.set_ylabel('f(xₖ, yₖ)', fontsize=12)
    ax1.set_title('Évolution de la fonction objectif', fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='r', linestyle='--', alpha=0.5, label='Minimum global')
    ax1.legend()
    
    # Graphique 2 : Évolution de la norme du gradient
    ax2.semilogy(iterations, grad_norms, 'g-', linewidth=2)
    ax2.set_xlabel('Itération k', fontsize=12)
    ax2.set_ylabel('||∇f(xₖ, yₖ)||', fontsize=12)
    ax2.set_title('Évolution de la norme du gradient', fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


def print_iteration_table(trajectory, n_rows=15):
    """
    Affiche un tableau détaillé des premières et dernières itérations.
    """
    print("=" * 100)
    print("TABLEAU DES ITÉRATIONS")
    print("=" * 100)
    
    # En-tête du tableau
    header = f"{'k':>5} | {'xₖ':>12} | {'yₖ':>12} | {'f(xₖ,yₖ)':>15} | {'||∇f||':>15}"
    
    # Vérifier si on a le pas optimal dans les données
    if 's_k' in trajectory[0]:
        header += f" | {'sₖ':>12}"
    
    print(header)
    print("-" * 100)
    
    # Afficher les n_rows premières itérations
    n_display = min(n_rows, len(trajectory))
    for point in trajectory[:n_display]:
        row = f"{point['iter']:>5} | {point['x']:>12.6f} | {point['y']:>12.6f} | {point['f']:>15.6e} | {point['grad_norm']:>15.6e}"
        if 's_k' in point and point['s_k'] is not None:
            row += f" | {point['s_k']:>12.6f}"
        print(row)
    
    # Si la trajectoire est longue, afficher ...
    if len(trajectory) > 2 * n_rows:
        print("  ...  |     ...      |     ...      |      ...        |      ...        ")
        
        # Afficher les n_rows dernières itérations
        for point in trajectory[-n_rows:]:
            row = f"{point['iter']:>5} | {point['x']:>12.6f} | {point['y']:>12.6f} | {point['f']:>15.6e} | {point['grad_norm']:>15.6e}"
            if 's_k' in point and point['s_k'] is not None:
                row += f" | {point['s_k']:>12.6f}"
            print(row)
    
    print("=" * 100)
    
    # Afficher un résumé final
    final_point = trajectory[-1]
    print(f"\nRÉSUMÉ:")
    print(f"  Nombre total d'itérations : {final_point['iter']}")
    print(f"  Point final : ({final_point['x']:.8f}, {final_point['y']:.8f})")
    print(f"  Valeur finale : f = {final_point['f']:.6e}")
    print(f"  Norme du gradient final : ||∇f|| = {final_point['grad_norm']:.6e}")
    print(f"  Erreur au minimum : |x-1| = {abs(final_point['x']-1):.6e}, |y-1| = {abs(final_point['y']-1):.6e}")
    print("=" * 100)

print("Fonctions de visualisation définies !")

#### **6. Expérimentation : Gradient à Pas Optimal**

#### Nous allons maintenant exécuter la méthode de plus profonde descente à partir du point initial $(-1, 2)$.


In [ ]:
#Point de départ
x0, y0 = -1, 2

print("="*90)
print("EXPÉRIMENTATION 1 : MÉTHODE DE GRADIENT À PAS OPTIMAL")
print("="*90)
print(f"\nPoint de départ : ({x0}, {y0})")
# Vérifier que les fonctions sont définies
try:
    rosenbrock(0, 0)
    print("✓ Fonctions correctement définies")
except NameError:
    print("✗ ERREUR : Exécutez d'abord les cellules de définition des fonctions !")
print(f"Valeur initiale : f({x0}, {y0}) = {rosenbrock(x0, y0):.6f}")
print("\nExécution de l'algorithme...\n")

# Exécuter l'algorithme
traj_optimal = gradient_pas_optimal(x0, y0, epsilon=1e-5, max_iter=10000)

#### **6.1 Tableau des itérations**

In [ ]:
print_iteration_table(traj_optimal, n_rows=10)

#### **6.2 Visualisation de la trajectoire**

In [ ]:
plot_contours_with_trajectory(traj_optimal, 
                              "Rosenbrock : Gradient à pas optimal (Steepest Descent)")


#### **6.3 Courbes de convergence**

In [ ]:
plot_convergence(traj_optimal, "Rosenbrock : Convergence avec pas optimal")

#### **6.4 Observations sur le phénomène de zigzag**

#### Comme observé dans le cours (pages 32-34), la méthode de plus profonde descente présente un comportement en zigzag. 
 
#### Vérifions l'orthogonalité des gradients successifs :


In [ ]:
print("Vérification de l'orthogonalité des gradients successifs :")
print("="*70)
print(f"{'Itération k':>12} | {'⟨∇f(xₖ), ∇f(xₖ₊₁)⟩':>25} | {'Orthogonaux?':>15}")
print("-"*70)

# Prendre les 10 premières itérations
for i in range(min(10, len(traj_optimal)-1)):
    point_k = traj_optimal[i]
    point_k1 = traj_optimal[i+1]
    
    grad_k = np.array(gradient_rosenbrock(point_k['x'], point_k['y']))
    grad_k1 = np.array(gradient_rosenbrock(point_k1['x'], point_k1['y']))
    
    produit_scalaire = np.dot(grad_k, grad_k1)
    orthogonal = "Oui" if abs(produit_scalaire) < 1e-6 else "Non"
    
    print(f"{i:>12} | {produit_scalaire:>25.6e} | {orthogonal:>15}")

print("="*70)
print("\nConclusion : Les gradients successifs sont (quasiment) orthogonaux,")
print("ce qui explique le comportement en zigzag de la trajectoire.")

#### **7. Expérimentation : Gradient à Pas Fixe**

#### Nous allons maintenant tester la méthode de gradient à pas fixe avec différentes valeurs du pas.

In [ ]:
def compare_fixed_steps(x0, y0, step_sizes, epsilon=1e-5, max_iter=100000):
    """
    Compare les performances de l'algorithme de gradient à pas fixe
    pour différentes valeurs du pas.
    """
    results = []
    
    print("\n" + "="*80)
    print("COMPARAISON DES DIFFÉRENTS PAS FIXES")
    print("="*80)
    print(f"{'Pas s':>12} | {'Converge?':>10} | {'Nb itérations':>15} | {'Valeur finale':>18} | {'||∇f|| final':>18}")
    print("-"*80)
    
    for s in step_sizes:
        trajectory, converged = gradient_pas_fixe(x0, y0, s, epsilon, max_iter)
        
        if converged:
            final = trajectory[-1]
            results.append({
                'step': s,
                'converged': True,
                'iterations': final['iter'],
                'final_value': final['f'],
                'final_grad_norm': final['grad_norm'],
                'trajectory': trajectory
            })
            print(f"{s:>12.6f} | {'Oui':>10} | {final['iter']:>15} | {final['f']:>18.6e} | {final['grad_norm']:>18.6e}")
        else:
            results.append({
                'step': s,
                'converged': False,
                'iterations': len(trajectory),
                'trajectory': trajectory
            })
            print(f"{s:>12.6f} | {'Non (DV)':>10} | {len(trajectory):>15} | {'Divergence':>18} | {'-':>18}")
    
    print("="*80)
    
    return results


#### **7.1 Test de différents pas**


In [ ]:
print("="*90)
print("EXPÉRIMENTATION 2 : MÉTHODE DE GRADIENT À PAS FIXE")
print("="*90)

# Tester une gamme de pas
pas_a_tester = [0.0001, 0.0003, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.01]

resultats_pas_fixes = compare_fixed_steps(x0, y0, pas_a_tester, 
                                          epsilon=1e-5, max_iter=100000)


#### **7.2 Visualisation du meilleur cas**

In [ ]:
# Trouver le meilleur cas (celui qui converge le plus rapidement)
cas_convergents = [r for r in resultats_pas_fixes if r['converged']]

if cas_convergents:
    meilleur_cas = min(cas_convergents, key=lambda x: x['iterations'])
    print(f"\n{'='*90}")
    print(f"MEILLEUR PAS FIXE : s = {meilleur_cas['step']}")
    print(f"{'='*90}")
    print(f"Nombre d'itérations : {meilleur_cas['iterations']}")
    print(f"Ratio par rapport au pas optimal : {meilleur_cas['iterations'] / traj_optimal[-1]['iter']:.2f}")
    
    # Tableau
    print_iteration_table(meilleur_cas['trajectory'], n_rows=10)
    
    # Visualisation
    plot_contours_with_trajectory(meilleur_cas['trajectory'],
                                  f"Rosenbrock : Gradient à pas fixe (s = {meilleur_cas['step']})")
    
    plot_convergence(meilleur_cas['trajectory'],
                    f"Rosenbrock : Convergence avec s = {meilleur_cas['step']}")


#### **7.3 Visualisation d'un cas plus lent**

In [ ]:
if len(cas_convergents) > 1:
    # Prendre un cas avec convergence plus lente (pas plus petit)
    cas_lents = [r for r in cas_convergents if r['step'] < meilleur_cas['step']]
    if cas_lents:
        cas_lent = min(cas_lents, key=lambda x: x['step'])
        print(f"\n{'='*90}")
        print(f"CAS AVEC PAS PLUS PETIT : s = {cas_lent['step']}")
        print(f"{'='*90}")
        print(f"Nombre d'itérations : {cas_lent['iterations']}")
        print(f"Ratio par rapport au pas optimal : {cas_lent['iterations'] / traj_optimal[-1]['iter']:.2f}")
        
        plot_contours_with_trajectory(cas_lent['trajectory'],
                                      f"Rosenbrock : Gradient à pas fixe (s = {cas_lent['step']})")


#### **7.4 Visualisation d'un cas de divergence (si applicable)**


In [ ]:
cas_divergents = [r for r in resultats_pas_fixes if not r['converged']]

if cas_divergents:
    cas_div = cas_divergents[0]
    print(f"\n{'='*90}")
    print(f"CAS DE DIVERGENCE : s = {cas_div['step']}")
    print(f"{'='*90}")
    print(f"Divergence après {cas_div['iterations']} itérations")
    
    if len(cas_div['trajectory']) > 1:
        plot_contours_with_trajectory(cas_div['trajectory'],
                                      f"Rosenbrock : DIVERGENCE avec s = {cas_div['step']}")


#### **8. Comparaison Graphique : Pas Optimal vs Meilleurs Pas Fixes**

In [ ]:
if cas_convergents:
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Courbes de niveau
    x = np.linspace(-2, 2, 400)
    y = np.linspace(-1, 3, 400)
    X, Y = np.meshgrid(x, y)
    Z = rosenbrock(X, Y)
    levels = np.logspace(-1, 3.5, 35)
    contour = ax.contour(X, Y, Z, levels=levels, cmap='gray', alpha=0.3)
    
    # Trajectoire pas optimal
    traj_x_opt = [p['x'] for p in traj_optimal]
    traj_y_opt = [p['y'] for p in traj_optimal]
    ax.plot(traj_x_opt, traj_y_opt, 'b.-', linewidth=2, markersize=6, 
            label=f'Pas optimal ({traj_optimal[-1]["iter"]} iter)', alpha=0.7)
    
    # Trajectoire meilleur pas fixe
    traj_x_best = [p['x'] for p in meilleur_cas['trajectory']]
    traj_y_best = [p['y'] for p in meilleur_cas['trajectory']]
    ax.plot(traj_x_best, traj_y_best, 'r.-', linewidth=2, markersize=6,
            label=f'Pas fixe s={meilleur_cas["step"]} ({meilleur_cas["iterations"]} iter)', alpha=0.7)
    
    # Marques
    ax.plot(x0, y0, 'go', markersize=15, label='Départ', zorder=5)
    ax.plot(1, 1, 'r*', markersize=20, label='Minimum (1,1)', zorder=5)
    
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title('Comparaison : Pas Optimal vs Pas Fixe', fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

#### **9. Visualisation 3D (Bonus)**

In [ ]:
def plot_3d_surface(trajectory=None):
    """
    Crée une visualisation 3D de la fonction de Rosenbrock.
    Si une trajectoire est fournie, elle est projetée sur la surface.
    """
    # Créer la grille
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-1, 3, 100)
    X, Y = np.meshgrid(x, y)
    Z = rosenbrock(X, Y)
    
    # Limiter Z pour une meilleure visualisation
    Z = np.minimum(Z, 500)
    
    # Créer la figure 3D
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Tracer la surface
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.7, 
                           edgecolor='none', antialiased=True)
    
    # Si une trajectoire est fournie, la tracer sur la surface
    if trajectory is not None:
        traj_x = np.array([point['x'] for point in trajectory])
        traj_y = np.array([point['y'] for point in trajectory])
        traj_z = np.array([point['f'] for point in trajectory])
        
        # Limiter les valeurs z pour la visualisation
        traj_z = np.minimum(traj_z, 500)
        
        ax.plot(traj_x, traj_y, traj_z, 'r-', linewidth=3, label='Trajectoire')
        ax.scatter(traj_x[0], traj_y[0], traj_z[0], c='green', s=100, 
                  marker='o', label='Départ', zorder=5)
        ax.scatter(traj_x[-1], traj_y[-1], traj_z[-1], c='blue', s=100, 
                  marker='o', label='Arrivée', zorder=5)
    
    # Marquer le minimum
    ax.scatter(1, 1, 0, c='red', s=200, marker='*', 
              label='Minimum global', zorder=5)
    
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_zlabel('f(x, y)', fontsize=12)
    ax.set_title('Fonction de Rosenbrock en 3D', fontsize=14, fontweight='bold')
    ax.legend()
    
    # Ajouter une barre de couleur
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
    
    plt.tight_layout()
    plt.show()

# Visualisation 3D avec la trajectoire du pas optimal
plot_3d_surface(traj_optimal)

#### **10. Analyse Détaillée des Résultats**

In [ ]:
def analyser_resultats_rosenbrock(resultats_pas_optimal, resultats_pas_fixes, x0, y0):
    """
    Génère une analyse complète et structurée des résultats d'optimisation
    pour la fonction de Rosenbrock.
    """
    
    print("\n" + "="*90)
    print(" "*25 + "ANALYSE DES RÉSULTATS")
    print(" "*20 + "Fonction de Rosenbrock")
    print("="*90)
    
    # ========== PARTIE 1 : PRÉSENTATION DE LA FONCTION ==========
    print("\n" + "-"*90)
    print("1. CARACTÉRISTIQUES DE LA FONCTION DE ROSENBROCK")
    print("-"*90)
    
    print("""
La fonction de Rosenbrock, également appelée fonction "banane" ou "vallée de Rosenbrock",
est définie par :

    f(x, y) = (1 - x)² + 100(y - x²)²

Cette fonction est un cas test classique en optimisation numérique car elle présente
plusieurs propriétés intéressantes et difficiles à gérer pour les algorithmes d'optimisation.

Propriétés principales :
    • Minimum global unique en (x*, y*) = (1, 1) avec f(1, 1) = 0
    • Fonction non-convexe (contrairement aux exemples du cours)
    • Vallée étroite et incurvée en forme de banane menant au minimum
    • Gradient faible le long de la vallée mais très raide perpendiculairement
    • La convergence est difficile car il faut d'abord trouver la vallée, 
      puis la suivre sur une longue distance jusqu'au minimum

Ces caractéristiques rendent la fonction de Rosenbrock particulièrement intéressante
pour tester la robustesse et l'efficacité des algorithmes d'optimisation.
""")
    
    print(f"Point de départ utilisé : ({x0}, {y0})")
    print(f"Valeur initiale de la fonction : f({x0}, {y0}) = {rosenbrock(x0, y0):.6e}")
    
    # ========== PARTIE 2 : ANALYSE DE LA MÉTHODE À PAS OPTIMAL ==========
    print("\n" + "-"*90)
    print("2. ANALYSE DE LA MÉTHODE DE PLUS PROFONDE DESCENTE (PAS OPTIMAL)")
    print("-"*90)
    
    n_iter_optimal = resultats_pas_optimal[-1]['iter']
    valeur_finale_optimal = resultats_pas_optimal[-1]['f']
    point_final_optimal = (resultats_pas_optimal[-1]['x'], resultats_pas_optimal[-1]['y'])
    
    print(f"""
Cette méthode calcule à chaque itération le pas optimal sₖ qui minimise
f(xₖ - s∇f(xₖ)) par rapport à s. C'est la méthode idéalisée du gradient.

Résultats obtenus :
    • Nombre d'itérations : {n_iter_optimal}
    • Point final : ({point_final_optimal[0]:.8f}, {point_final_optimal[1]:.8f})
    • Valeur finale : f = {valeur_finale_optimal:.6e}
    • Erreur au minimum : |x-1| = {abs(point_final_optimal[0]-1):.6e}
                          |y-1| = {abs(point_final_optimal[1]-1):.6e}
""")
    
    # Analyser le phénomène de zigzag
    print("Observation du phénomène de zigzag :")
    print("""
Comme observé dans le cours (pages 32-34) pour la fonction quadratique, la méthode
de plus profonde descente présente un comportement caractéristique en zigzag.
Ce phénomène s'explique par une propriété mathématique fondamentale : deux directions
de descente successives sont orthogonales.

En effet, si sₖ est le pas optimal à l'itération k, alors il vérifie la condition
d'optimalité du premier ordre :

    d/ds[f(xₖ - s∇f(xₖ))]|ₛ=ₛₖ = 0

Ce qui implique que : ⟨∇f(xₖ), ∇f(xₖ₊₁)⟩ = 0

Cette orthogonalité des gradients successifs se traduit visuellement par un mouvement
en zigzag où l'algorithme effectue des allers-retours perpendiculaires dans la vallée,
plutôt que de la suivre directement. Ce comportement est particulièrement prononcé
pour la fonction de Rosenbrock en raison de sa vallée étroite et courbée.
""")
    
    # Analyser quelques pas optimaux
    if len(resultats_pas_optimal) > 10:
        pas_optimaux = [p['s_k'] for p in resultats_pas_optimal[1:11] if p['s_k'] is not None]
        if pas_optimaux:
            print(f"Variation des pas optimaux (10 premières itérations) :")
            print(f"    • Minimum : {min(pas_optimaux):.6f}")
            print(f"    • Maximum : {max(pas_optimaux):.6f}")
            print(f"    • Moyenne : {np.mean(pas_optimaux):.6f}")
            print("""
La variation importante des pas optimaux montre que l'algorithme s'adapte à la
géométrie locale de la fonction. Dans les zones raides, le pas est petit, tandis
que dans les zones plus plates, le pas peut être plus grand.
""")
    
    # ========== PARTIE 3 : ANALYSE DES MÉTHODES À PAS FIXE ==========
    print("\n" + "-"*90)
    print("3. ANALYSE DES MÉTHODES À PAS FIXE")
    print("-"*90)
    
    print("""
La méthode à pas fixe est plus simple à implémenter car elle ne nécessite pas
de résoudre un problème d'optimisation unidimensionnel à chaque itération.
Cependant, le choix du pas est critique pour garantir la convergence.
""")
    
    # Séparer les résultats convergés et divergés
    converges = [r for r in resultats_pas_fixes if r['converged']]
    diverges = [r for r in resultats_pas_fixes if not r['converged']]
    
    if converges:
        print(f"\nCas de convergence ({len(converges)} pas testés) :")
        print("-" * 80)
        
        meilleur = min(converges, key=lambda x: x['iterations'])
        
        for res in sorted(converges, key=lambda x: x['step']):
            ratio = res['iterations'] / n_iter_optimal if n_iter_optimal > 0 else float('inf')
            print(f"    • Pas s = {res['step']:.6f} :")
            print(f"        - {res['iterations']} itérations (×{ratio:.2f} par rapport au pas optimal)")
            print(f"        - Valeur finale : {res['final_value']:.6e}")
            print(f"        - Point final : ({res['trajectory'][-1]['x']:.6f}, {res['trajectory'][-1]['y']:.6f})")
        
        print(f"""
Observation : Le meilleur pas fixe testé est s = {meilleur['step']:.6f} avec 
{meilleur['iterations']} itérations.
""")
        
        print("""
Analyse de l'influence du pas :

Pour la fonction de Rosenbrock, nous observons que :

    1. Les pas très petits (s << 0.001) convergent mais très lentement.
       L'algorithme progresse à tout petits pas et nécessite un très grand nombre
       d'itérations. C'est une approche sûre mais inefficace.
    
    2. Il existe une plage de pas "optimaux" où la convergence est relativement
       rapide. Ces pas permettent de faire des progrès significatifs à chaque
       itération sans risquer de divergence.
    
    3. Les pas trop grands (s > seuil_critique) font diverger l'algorithme.
       Les itérations "sautent" par-dessus le minimum et s'éloignent de plus
       en plus de la solution.

La théorie de la convergence nous enseigne que pour garantir la convergence
d'une méthode de gradient à pas fixe, il faut que le pas vérifie :

    0 < s < 2/L

où L est la constante de Lipschitz du gradient (liée à la plus grande valeur
propre de la Hessienne). Pour Rosenbrock, cette constante varie avec la position,
ce qui explique pourquoi certains pas fonctionnent et d'autres non.
""")
    
    if diverges:
        print(f"\nCas de divergence ({len(diverges)} pas testés) :")
        print("-" * 80)
        
        for res in sorted(diverges, key=lambda x: x['step']):
            print(f"    • Pas s = {res['step']:.6f} : Divergence après {res['iterations']} itérations")
        
        print("""
Ces divergences illustrent l'importance cruciale du choix du pas dans les méthodes
à pas fixe. Un pas trop grand viole les conditions de convergence théoriques et
l'algorithme s'éloigne du minimum au lieu de s'en rapprocher.
""")
    
    # ========== PARTIE 4 : COMPARAISON GLOBALE ==========
    print("\n" + "-"*90)
    print("4. COMPARAISON GLOBALE ET RECOMMANDATIONS")
    print("-"*90)
    
    print("""
Synthèse des observations :
""")
    
    print(f"""
Méthode à pas optimal :
    • Avantages : 
        - Convergence garantie (sous certaines hypothèses)
        - Pas besoin de régler de paramètre
        - Progression "optimale" à chaque itération
    • Inconvénients :
        - Comportement en zigzag inefficace dans les vallées étroites
        - Coût élevé : nécessite de résoudre un problème d'optimisation 1D 
          à chaque itération
        - {n_iter_optimal} itérations pour cette fonction
""")
    
    if converges:
        print(f"""
Méthode à pas fixe (meilleur cas : s = {meilleur['step']:.6f}) :
    • Avantages :
        - Simplicité d'implémentation
        - Coût par itération minimal
        - Peut être plus rapide que le pas optimal si s est bien choisi
    • Inconvénients :
        - Nécessite de choisir le bon pas (essais-erreurs ou théorie)
        - Risque de divergence si s est trop grand
        - Convergence lente si s est trop petit
        - {meilleur['iterations']} itérations pour le meilleur pas testé
""")
    
    print("""
Recommandations pratiques :

Pour la fonction de Rosenbrock et les fonctions similaires (non-convexes avec
vallées étroites), les méthodes de gradient simple (pas optimal ou pas fixe)
ne sont pas les plus efficaces. Les observations suivantes sont importantes :

    1. Le phénomène de zigzag ralentit considérablement la convergence dans
       les vallées étroites. Des méthodes plus sophistiquées comme Newton ou
       quasi-Newton (BFGS) sont préférables car elles tiennent compte de la
       courbure (Hessienne) et peuvent "suivre" la vallée plus efficacement.
    
    2. Pour le gradient à pas fixe, le choix du pas est délicat et dépend
       de la région de l'espace où l'on se trouve. Une stratégie de pas
       adaptatif serait plus robuste.
    
    3. La convergence théorique (||∇f|| → 0) ne garantit pas qu'on atteigne
       le minimum global pour des fonctions non-convexes. Le point initial
       est donc crucial.

Ces limitations des méthodes de gradient justifient l'étude des méthodes
plus avancées vues dans le cours (Newton, quasi-Newton, gradient conjugué).
""")
    
    print("="*90 + "\n")
    
    return {
        'n_iter_optimal': n_iter_optimal,
        'meilleur_pas_fixe': meilleur if converges else None,
        'pas_divergents': [r['step'] for r in diverges]
    }

# Exécuter l'analyse complète
analyse = analyser_resultats_rosenbrock(traj_optimal, resultats_pas_fixes, x0, y0)


#### **11. Graphique de Synthèse : Nombre d'itérations vs Taille du pas**

In [ ]:
if cas_convergents:
    # Préparer les données
    pas_values = [r['step'] for r in cas_convergents]
    iter_values = [r['iterations'] for r in cas_convergents]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Tracer le graphique
    ax.plot(pas_values, iter_values, 'bo-', linewidth=2, markersize=8, label='Pas fixe')
    
    # Ligne horizontale pour le pas optimal
    ax.axhline(y=traj_optimal[-1]['iter'], color='r', linestyle='--', 
               linewidth=2, label=f'Pas optimal ({traj_optimal[-1]["iter"]} iter)')
    
    ax.set_xlabel('Taille du pas (s)', fontsize=12)
    ax.set_ylabel('Nombre d\'itérations', fontsize=12)
    ax.set_title('Influence de la taille du pas sur la convergence', 
                 fontsize=14, fontweight='bold')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    print("\nObservation :")
    print("-" * 80)
    print("Le graphique montre clairement la relation entre la taille du pas et")
    print("le nombre d'itérations nécessaires pour converger :")
    print("  • Pas très petit → beaucoup d'itérations (convergence lente)")
    print("  • Pas optimal intermédiaire → nombre d'itérations réduit")
    print("  • Pas trop grand → divergence (non représenté)")


#### **12. Conclusion**

#### **Résumé des résultats**

#### Dans ce notebook, nous avons étudié l'optimisation de la fonction de Rosenbrock avec deux méthodes de gradient :

#### 1. **Gradient à pas optimal** : Convergence en {n_iter_optimal} itérations mais avec un comportement en zigzag caractéristique

#### 2. **Gradient à pas fixe** : Les performances varient fortement selon la taille du pas :
####    - Pas trop petit : convergence très lente
####    - Pas bien choisi : performances comparables au pas optimal
####    - Pas trop grand : divergence
###
#### **Observations principales**

#### - Le phénomène de **zigzag** observé avec le pas optimal est dû à l'orthogonalité des gradients successifs
#### - La **vallée étroite** de Rosenbrock rend la convergence difficile pour les méthodes de gradient simple
#### - Le choix du **pas fixe** est crucial et dépend de la géométrie locale de la fonction
#### - Ces méthodes simples sont **inefficaces** pour ce type de fonction, justifiant l'usage de méthodes plus avancées
###
#### **Perspectives**

#### Pour améliorer les performances sur la fonction de Rosenbrock, il faudrait utiliser :
#### - La méthode de **Newton** qui utilise la Hessienne
#### - Les méthodes **quasi-Newton** (BFGS, L-BFGS)
#### - Le **gradient conjugué**
#### - Des stratégies de **pas adaptatif** (Armijo, Wolfe)
####
#### Ces méthodes sont étudiées dans la suite du cours et permettent de mieux gérer les vallées étroites.

#
# ---
### **Fin du notebook pour la fonction de Rosenbrock**

#### Dans les prochains notebooks, nous appliquerons la même méthodologie aux fonctions :
#### - Quadratique : $f(x, y) = x^2 - y^2$
#### - Himmelblau : $f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2$